In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
ranking_task = mz.tasks.Ranking(losses=mz.losses.RankCrossEntropyLoss(num_neg=10))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [3]:
preprocessor = mz.models.MatchLSTM.get_default_preprocessor()

In [4]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 9996.95it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5446.54it/s]
Building Vocabulary from a datapack.: 100%|██████████| 418401/418401 [00:00<00:00, 3278129.97it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 10414.40it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:03<00:00, 5462.16it/s]
Processing length_right with len: 100%|██████████| 18841/18841 [00:00<00:00, 1012581.29it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 10083.06it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [5]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f87387db160>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f87387673c8>,
 'vocab_size': 30058,
 'embedding_input_dim': 30058}

In [6]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
term_index = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = glove_embedding.build_matrix(term_index)
l2_norm = np.sqrt((embedding_matrix * embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [7]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=5,
    num_neg=10
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [8]:
padding_callback = mz.models.MatchLSTM.get_default_padding_callback()

trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    shuffle=True,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    sort=False,
    shuffle=False,
    callback=padding_callback
)

In [9]:
model = mz.models.MatchLSTM()

model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding'] = embedding_matrix

model.build()

print(model, sum(p.numel() for p in model.parameters() if p.requires_grad))

MatchLSTM(
  (embedding): Embedding(30058, 300)
  (dropout): Dropout(p=0.2, inplace=False)
  (input_proj): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(300, 100, bidirectional=True)
    )
  )
  (match_module): MatchModule(
    (v2_proj): Linear(in_features=200, out_features=200, bias=True)
    (proj): Linear(in_features=800, out_features=400, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (mlstm_module): StackedBRNN(
    (rnns): ModuleList(
      (0): LSTM(400, 100, bidirectional=True)
    )
  )
  (classification): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=200, out_features=200, bias=True)
    (2): Tanh()
  )
  (out): Linear(in_features=200, out_features=1, bias=True)
) 10141601


In [10]:
optimizer = torch.optim.Adadelta(model.parameters())

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [11]:
trainer.run()

[Iter-255 Loss-1.887]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5723 - normalized_discounted_cumulative_gain@5(0.0): 0.631 - mean_average_precision(0.0): 0.5943



[Iter-510 Loss-0.495]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5258 - normalized_discounted_cumulative_gain@5(0.0): 0.5779 - mean_average_precision(0.0): 0.5407



[Iter-765 Loss-0.098]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5199 - normalized_discounted_cumulative_gain@5(0.0): 0.574 - mean_average_precision(0.0): 0.5296



[Iter-1020 Loss-0.050]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5416 - normalized_discounted_cumulative_gain@5(0.0): 0.5945 - mean_average_precision(0.0): 0.5525



[Iter-1275 Loss-0.041]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5127 - normalized_discounted_cumulative_gain@5(0.0): 0.577 - mean_average_precision(0.0): 0.5347



[Iter-1530 Loss-0.031]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5121 - normalized_discounted_cumulative_gain@5(0.0): 0.575 - mean_average_precision(0.0): 0.5299



[Iter-1785 Loss-0.028]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5128 - normalized_discounted_cumulative_gain@5(0.0): 0.5846 - mean_average_precision(0.0): 0.5359



[Iter-2040 Loss-0.026]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5084 - normalized_discounted_cumulative_gain@5(0.0): 0.5746 - mean_average_precision(0.0): 0.5259



[Iter-2295 Loss-0.021]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5188 - normalized_discounted_cumulative_gain@5(0.0): 0.5885 - mean_average_precision(0.0): 0.5439



[Iter-2550 Loss-0.015]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5098 - normalized_discounted_cumulative_gain@5(0.0): 0.5786 - mean_average_precision(0.0): 0.5357

Cost time: 8400.133549451828s
